# Notebook 01: Exploratory Data Analysis and Reporting

This notebook performs a detailed analysis of the cleaned Home Credit dataset, including per-variable summaries, target-stratified analyses, default rate per category, and correlation visualization. Methodology aligns with best practices as described in Harrell (2015) and is modeled after Berg et al. (2020).


**Key EDA References:**

- Harrell, F. E. (2015). Regression Modeling Strategies. Springer. (See Chapters 3 and 4 for recommended variable-by-variable reporting and stratification by outcome.)
- Berg, T., Burg, V., Gombović, A., & Puri, M. (2020). On the Rise of FinTechs: Credit Scoring Using Digital Footprints. Review of Financial Studies, 33(7), 2845–2897. (See their Table 2 and reporting methodology.)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned, subsampled dataset
df = pd.read_csv('/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/data/processed/home_credit_sample.csv')


## General summary statistics

Overview for numeric and categorical variables.


In [4]:
summary = df.describe(include='all').T
summary['missing_pct'] = df.isnull().mean()
display(summary)
summary.to_csv('/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/tables/reporting_overview_summary.csv')


,count,unique,top,freq,mean,std,min,25%,50%,75%,max,missing_pct
EXT_SOURCE_1,5000.0,NaN,NaN,NaN,0.503611,0.139889,0.039579,0.505998,0.505998,0.505998,0.931751,0.0
EXT_SOURCE_2,5000.0,NaN,NaN,NaN,0.511512,0.190548,0.000085,0.387881,0.56425,0.66138,0.809908,0.0
EXT_SOURCE_3,5000.0,NaN,NaN,NaN,0.514711,0.17291,0.000527,0.411849,0.535276,0.633032,0.885488,0.0
AMT_CREDIT,5000.0,NaN,NaN,NaN,604635.183,406377.614999,45000.0,270000.0,518562.0,810000.0,2700000.0,0.0
AMT_ANNUITY,5000.0,NaN,NaN,NaN,27331.3899,14623.093876,3145.5,16573.5,25328.25,34756.875,130018.5,0.0
AMT_GOODS_PRICE,5000.0,NaN,NaN,NaN,543855.5865,371623.026268,45000.0,238500.0,450000.0,679500.0,2700000.0,0.0
DAYS_BIRTH,5000.0,NaN,NaN,NaN,-15952.7762,4409.809171,-25142.0,-19683.0,-15715.0,-12141.0,-7680.0,0.0
CODE_GENDER,5000,2,F,3338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
CNT_CHILDREN,5000.0,NaN,NaN,NaN,0.428,0.734387,0.0,0.0,0.0,1.0,5.0,0.0
CNT_FAM_MEMBERS,5000.0,NaN,NaN,NaN,2.1668,0.917794,1.0,2.0,2.0,3.0,6.0,0.0


## Numeric variable distributions (overall)


In [6]:
numeric_vars = df.select_dtypes(include=[np.number]).columns.drop('TARGET')
for col in numeric_vars:
    plt.figure(figsize=(6,3))
    sns.histplot(df[col], bins=30, kde=True)
    plt.title(f'Distribution: {col}')
    plt.savefig(f'/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/figures/{col}_hist.png')
    plt.close()


## Default rate by category for each categorical variable

For every categorical variable (e.g., education, gender), show each value, count, and the default rate (`TARGET=1`).


In [16]:
import os
os.makedirs('/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/tables/default_rate_per_variable', exist_ok=True)

cat_vars = df.select_dtypes(include=['object', 'category']).columns

# Save one CSV per categorical variable
for col in cat_vars:
    table = df.groupby(col)['TARGET'].agg(['count', 'mean']).reset_index()
    table.rename(columns={'mean': 'default_rate'}, inplace=True)
    table['default_rate'] = table['default_rate'].round(4)
    table.to_csv(f'/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/tables/default_rate_per_variable/{col}_default_rate.csv', index=False)


## Single summary table: all categorical variable default rates

Combine all (variable, value, default rate) info into one big CSV for review.


In [17]:
cat_default_list = []
for col in cat_vars:
    table = df.groupby(col)['TARGET'].agg(['count', 'mean']).reset_index()
    table['variable'] = col
    table.rename(columns={col: 'value', 'mean': 'default_rate'}, inplace=True)
    table['default_rate'] = table['default_rate'].round(4)
    cat_default_list.append(table[['variable', 'value', 'count', 'default_rate']])
cat_default_summary = pd.concat(cat_default_list, ignore_index=True)
cat_default_summary.to_csv('/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/tables/default_rate_per_variable/categorical_default_summary.csv', index=False)
cat_default_summary.head()


,variable,value,count,default_rate
0,CODE_GENDER,F,3338,0.0725
1,CODE_GENDER,M,1662,0.0975
2,NAME_EDUCATION_TYPE,Academic degree,1,0.0000
3,NAME_EDUCATION_TYPE,Higher education,1176,0.0493
4,NAME_EDUCATION_TYPE,Incomplete higher,190,0.0737


## Default rate by value bin for numeric variables

Bin each numeric variable (quartiles), show default rate per bin.


In [ ]:
import numpy as np

numeric_vars = df.select_dtypes(include=[np.number]).columns.drop('TARGET')
numeric_default_list = []
for col in numeric_vars:
    try:
        # Use 4 quantile bins (quartiles)
        df[f'{col}_bin'] = pd.qcut(df[col], q=4, duplicates='drop')
        table = df.groupby(f'{col}_bin')['TARGET'].agg(['count', 'mean']).reset_index()
        table['variable'] = col
        table.rename(columns={'mean': 'default_rate', f'{col}_bin': 'bin'}, inplace=True)
        table['default_rate'] = table['default_rate'].round(4)
        numeric_default_list.append(table[['variable', 'bin', 'count', 'default_rate']])
        # Save per-variable as well
        table.to_csv(f'/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/tables/default_rate_per_variable/{col}_binned_default_rate.csv', index=False)
    except Exception as e:
        print(f"Could not bin {col}: {e}")

# All in one file
if numeric_default_list:
    numeric_default_summary = pd.concat(numeric_default_list, ignore_index=True)
    numeric_default_summary.to_csv('/home/frederickerleigh/Dokumente/Fintech Seminar/NewCode/FintechSeminar-Synthetic-Dataset/fintech-credit-scoring-seminar/results/tables/default_rate_per_variable/numeric_binned_default_summary.csv', index=False)
    numeric_default_summary.head()
